In [1]:
#import necessary libraries
import sempy.fabric as fabric
import pandas as pd
import numpy as np
import re
from notebookutils import mssparkutils
from datetime import date,datetime
import time
#from pyspark.sql.functions import col, to_date

#import sempy_labs as labs
#from sempy_labs import directlake
#from sempy_labs.tom import connect_semantic_model

###########################################################################################################
### Set Parameters
###########################################################################################################
#Define lakehouse and schema names
workspace_name = "amaser_dp700" #this is the workspace name where the notebook runs, and where the lakehouse lives
lakehouse_name = "test_fit" #this is the name of the lakehouse, in the event that it needs to becreated by this notebook
schema_name = "fitschema"  #This is the name of the schema.  if using non-schema-enabled lakehouse, this MUST be dbo
semantic_model_name = "FabricInventory" ###NOTE: IF THIS MODEL ALREADY EXISTS, IT WILL BE OVERWRITTEN

StatementMeta(, fe5d2027-5ec5-4f1b-b950-0f5fc334a6ac, 32, Finished, Available, Finished)

In [2]:
# Define variables and functions
full_lakehouse_name = f'{workspace_name}.{lakehouse_name}.{schema_name}'
current_date = date.today()
print(current_date)
###################################################################################################
### DEFINE FUNCTIONS
###################################################################################################
def logActivity(table_name,message):
    logquery = f'INSERT INTO {full_lakehouse_name}.inventorylog (timeval,workspace_id,Message) VALUES (CURRENT_TIMESTAMP(),"{table_name}","{message}")'
    #print(f'log query={logquery}')
    #print(logquery)
    spark.sql(logquery)

StatementMeta(, fe5d2027-5ec5-4f1b-b950-0f5fc334a6ac, 33, Finished, Available, Finished)

2026-01-06


In [ ]:

#Create date table
current_year = datetime.now().year
end_year = current_year + 1
#print(end_year)
def create_date_table(start_date='2018-01-01', end_date=f'{end_year}-12-31'):
    """
    Creates a Pandas DataFrame representing a full date table.

    Args:
        start_date (str): The start date for the date table (format 'YYYY-MM-DD').
        end_date (str): The end date for the date table (format 'YYYY-MM-DD').

    Returns:
        pd.DataFrame: A DataFrame containing various date-related columns.
    """
    df = pd.DataFrame({"Date": pd.date_range(start_date, end_date)})

    df["Day"] = df.Date.dt.day_name()
    df["DayOfWeek"] = df.Date.dt.dayofweek + 1 # Monday=1, Sunday=7
    df["DayOfMonth"] = df.Date.dt.day
    df["DayOfYear"] = df.Date.dt.dayofyear

    df["Week"] = df.Date.dt.isocalendar().week.astype(int)
    df["WeekOfYear"] = df.Date.dt.isocalendar().week.astype(int) # Same as Week

    df["Month"] = df.Date.dt.month
    df["MonthName"] = df.Date.dt.month_name()
    df["Quarter"] = df.Date.dt.quarter
    df["Year"] = df.Date.dt.year
    df["YearQuarter"] = df.Date.dt.strftime('%Y-Q%q')
    df["Year-Month"] = df.Date.dt.strftime('%Y-%m')

    df["IsWeekend"] = df.Date.dt.dayofweek.isin([5, 6]) # Saturday=5, Sunday=6
    df["IsLeapYear"] = df.Date.dt.is_leap_year

    return df

spark.sql(f"DROP TABLE IF EXISTS {workspace_name}.{lakehouse_name}.{schema_name}.DimDate")
# Create the date table
date_table_df = create_date_table()
print("DimDate table dropped")
# Display the first few rows of the DataFrame
#display(date_table_df.head())
#df["Year-Month"] = df["Year"] + "-" + df["Month"]
#print(df)

spark_df = spark.createDataFrame(date_table_df)
spark_df.write.mode("overwrite").format("delta").saveAsTable(f"{full_lakehouse_name}.DimDate")
print("DiMDate table created")

StatementMeta(, fe5d2027-5ec5-4f1b-b950-0f5fc334a6ac, 34, Finished, Available, Finished)

DimDate table dropped


DiMDate table created


In [3]:
print("Starting Workspace table load....")
logActivity("","Starting workspace table load")
all_workspaces_df = fabric.list_workspaces()
workspace_table_name = f'{full_lakehouse_name}.workspaces'
#print (all_workspaces_df.head())
workspaces_df = all_workspaces_df[['Name','Id','Is Read Only','Is On Dedicated Capacity','Capacity Id','Type']]
#print(workspaces_df)

delete_workspace_qeuery = f"DELETE FROM {workspace_table_name} WHERE timeval = '{current_date}'"
#delete_workspace_qeuery = f"DELETE FROM {workspace_table_name} "
print("Deleting workspace entries for the current date....")
logActivity("","Deleting workspace entries for the current date...")
spark.sql(delete_workspace_qeuery)
print("current date Workspace entries deleted.")
logActivity("","current date workspace entries deleted.")
#insert rows into workspace table
for index,row in workspaces_df.iterrows():
    row_sql = f"""
    INSERT INTO {workspace_table_name} (timeval,workspace_name, workspace_id,is_read_only,is_dedicated_capacity,capacity_id,type) 
    VALUES ('{current_date}', '{row['Name']}','{row['Id']}',{row['Is Read Only']},{row['Is On Dedicated Capacity']},'{row['Capacity Id']}','Workspace')
    """
    spark.sql(row_sql)
print("Workspaces inserted")
logActivity("","Workspaces inserted")

StatementMeta(, fe5d2027-5ec5-4f1b-b950-0f5fc334a6ac, 35, Finished, Available, Finished)

Starting Workspace table load....


Deleting workspace entries....


Workspace entries deleted.


Workspaces inserted


In [4]:
print("Starting fabricItems....")
logActivity("","Starting fabricItems")
items_table_name = f'{full_lakehouse_name}.fabricitems'
print("Deleting current date fabric item entries....")
logActivity("","Deleting current date fabric item entriess")
del_items_query = f"DELETE FROM {items_table_name} WHERE timeval = '{current_date}'"
spark.sql(del_items_query)
#display(workspaces_df)
rows = workspaces_df[["Name","Id"]]
#display(rows)
for index,row in rows.iterrows():
#for ws in rows:
    ws = row['Name']
    #print(ws)
    items_df = fabric.list_items(workspace=ws)
    
    items_df.rename(columns={'Display Name':'item_name','Workspace Id':'workspace_id','Id':'item_id','Type':'item_type'},inplace=True)
    
    del items_df['Folder Id']
   
    items_df['timeval'] = current_date
    #print(items_df)
    if items_df.empty:
        print(f"Cannot read data from {ws}")

        logActivity(f'{row["Id"]}',f"Cannot read data from {ws}")
    else:
        items_df = pd.merge(items_df,workspaces_df[['Name','Id']],left_on='workspace_id', right_on='Id',how='left')
        items_df = items_df.rename(columns={'Name':'workspace_name'})
        del items_df['Id']
        spark_df = spark.createDataFrame(items_df)
        spark_df.write.mode("append").format("delta").saveAsTable(items_table_name)
        print(f'{ws} fabric items entered')
        logActivity(f'{row["Id"]}',f'{ws} fabric items entered')
    
print("Current Date fabricItems inserted")
logActivity("","Current Date fabricItems inserted.")
     

StatementMeta(, fe5d2027-5ec5-4f1b-b950-0f5fc334a6ac, 36, Finished, Available, Finished)

Starting fabricItems....


Deleting current date fabric item entries....


DIF 1 LAB DEV fabric items entered


Workshop V2 DEV fabric items entered


...

amaser_dp700 fabric items entered


Industry Demos fabric items entered


PTurley AzureDevOps Integration fabric items entered


MDFL_Next_PTurley fabric items entered


Current Date fabricItems inserted


In [ ]:
time.sleep(120)
fabric.refresh_dataset(workspace=workspace_name, dataset=semantic_model_name)
print("Dataset refresh completed")
logActivity("","Dataset refresh completed")

StatementMeta(, fe5d2027-5ec5-4f1b-b950-0f5fc334a6ac, 37, Finished, Available, Finished)

Dataset refresh completed
